In [20]:
import tensorflow #Thư viện chính cho các nhiệm vụ deep learning.
import tensorflow as tf
#Các mô-đun này chứa các hàm và lớp để tạo các mô hình mạng thần kinh.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
#Chứa các chức năng tiện ích
from tensorflow.keras.utils import to_categorical
#Bao gồm các công cụ để xử lý trước dữ liệu, chẳng hạn như thay đổi kích thước và tăng kích thước hình ảnh.
from tensorflow.keras.preprocessing import image
#Các gói cơ bản cho tính toán khoa học và thao tác dữ liệu.
import numpy as np
import pandas as pd
#Mô hình VGG16 được đào tạo trước, được sử dụng để tận dụng việc học chuyển giao.
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import  Model
import matplotlib.pyplot as plt #Thư viện để tạo trực quan hóa tĩnh, tương tác và hoạt hình trong Python.
#Cụ thể, train_test_splitđược sử dụng để phân chia dữ liệu thành các tập huấn luyện và kiểm tra.
from sklearn.model_selection import train_test_split
from tqdm import tqdm #Thư viện hiển thị thanh tiến trình trong vòng lặp.
import cv2 #Thư viện OpenCV để xử lý ảnh.
from tensorflow.keras.preprocessing import image
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [21]:
#xác định các hằng số cho chiều rộng hình ảnh ( IMAGE_W) và chiều cao ( IMAGE_H), cả hai đều được đặt thành 224 pixel.
#Các kích thước này thường được sử dụng cho hình ảnh đầu vào trong các mô hình học sâu, đặc biệt là trong các mô hình dựa trên mạng thần kinh tích chập (CNN) được thiết kế cho các tác vụ nhận dạng hình ảnh.
IMAGE_W = 224
IMAGE_H = 224

In [22]:
#Import và tạo VGG16 base model
#VGG16: Là một mô hình kiến trúc mạng CNN (Convolutional Neural Network) sâu phổ biến.
#weights='imagenet': Sử dụng trọng số được huấn luyện trước trên tập dữ liệu ImageNet.
#include_top=False: Loại bỏ các lớp fully connected ở cuối mô hình, vì chúng ta sẽ tự định nghĩa các lớp này phù hợp với bài toán cụ thể của mình.
def get_model():
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

    # Dong bang cac layer
    # Đóng băng các lớp của mô hình VGG16
    # Điều này đảm bảo rằng các trọng số của mô hình VGG16 sẽ không được cập nhật trong quá trình huấn luyện. Mục tiêu là sử dụng mô hình này như một bộ trích xuất đặc trưng.
    for layer in model_vgg16_conv.layers:
        layer.trainable = False

    # Tao model
    # Tạo đầu vào và đầu ra của mô hình
    input = Input(shape=(IMAGE_W, IMAGE_H, 3), name='image_input') #Input: Tạo tensor đầu vào với kích thước ảnh (IMAGE_W, IMAGE_H, 3), ở đây 3 là số kênh màu.
    output_vgg16_conv = model_vgg16_conv(input) #output_vgg16_conv: Lấy đầu ra từ mô hình VGG16.

    # Them cac layer FC va Dropout
    # Thêm các lớp Fully Connected và Dropout
    x = Flatten(name='flatten')(output_vgg16_conv) #Flatten: Chuyển đổi output từ 3D sang 1D.
    x = Dense(4096, activation='relu', name='fc1')(x) #Dense: Các lớp fully connected với 4096 nút, sử dụng hàm kích hoạt relu.
    x = Dropout(0.5)(x) #Dropout: Ngăn chặn hiện tượng overfitting bằng cách ngẫu nhiên loại bỏ (drop) một số nút trong mạng.
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(5, activation='sigmoid', name='predictions')(x) #Lớp cuối cùng sử dụng hàm sigmoid cho mỗi nút vì đây là bài toán phân loại nhiều lớp (multi-label classification) với 5 lớp.

    # Compile
    my_model = Model(inputs=input, outputs=x)
    my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['BinaryAccuracy'])
    #binary_crossentropy: Hàm mất mát thích hợp cho bài toán phân loại nhị phân.
    #adam: Một thuật toán tối ưu hóa hiệu quả.
    #BinaryAccuracy: Độ chính xác nhị phân sẽ được sử dụng làm chỉ số đánh giá.
    return my_model

model = get_model() #hàm get_model() trả về mô hình đã được compile và sẵn sàng để huấn luyện. Mô hình này có thể được sử dụng để phân loại ảnh vào một trong năm lớp được định nghĩa.

In [23]:
csv_file = "/content/gdrive/MyDrive/Dataset/luanai_dataset/luanai_labels_1.csv"
image_path ="/content/gdrive/MyDrive/Dataset/luanai_dataset/images"
# Read the CSV file
train = pd.read_csv(csv_file)
train.head #train.head() trả về năm hàng đầu tiên của DataFrame

<bound method NDFrame.head of      Filenames  desert  mountains  sea  sunset  trees
0        1.jpg       1          0    0       0      0
1        2.jpg       1          0    0       0      0
2        3.jpg       1          0    0       0      0
3        4.jpg       1          1    0       0      0
4        5.jpg       1          0    0       0      0
...        ...     ...        ...  ...     ...    ...
1995  1996.jpg       0          0    0       0      1
1996  1997.jpg       0          0    0       0      1
1997  1998.jpg       0          0    0       0      1
1998  1999.jpg       0          0    0       0      1
1999  2000.jpg       0          0    0       0      1

[2000 rows x 6 columns]>

In [24]:
class Dataloader(tf.keras.utils.Sequence): #định nghĩa một lớp mới có tên Dataloader kế thừa từ lớp tf.keras.utils.Sequence trong thư viện TensorFlow/Keras.
    #Phương thức khởi tạo __init__ nhận ba tham số: dataset (tập dữ liệu), batch_size (kích thước batch), và size (kích thước tập dữ liệu).
    #Các tham số này được gán cho các thuộc tính self.dataset, self.batch_size, và self.size tương ứng.
    def __init__(self, dataset, batch_size, size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.size = size
    #Phương thức __getitem__ nhận một tham số i và trả về một batch dữ liệu tại vị trí index i.
    #Đầu tiên, nó tính toán chỉ số start và stop để xác định phạm vi của batch. Sau đó, nó duyệt qua phạm vi này và thêm các mẫu dữ liệu tương ứng vào danh sách data.
    def __getitem__(self, i):
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        #sử dụng np.stack để đóng gói các mẫu dữ liệu trong data thành các mảng numpy theo chiều axis=0. Cuối cùng, nó trả về một tuple chứa các mảng numpy này.
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        return tuple(batch)
    #Phương thức __len__ trả về số lượng batch trong tập dữ liệu bằng cách chia kích thước tập dữ liệu self.size cho kích thước batch self.batch_size.
    def __len__(self):
        return self.size // self.batch_size

class Dataset: #định nghĩa một lớp mới có tên Dataset
    #khởi tạo __init__ nhận bốn tham số: data (danh sách đường dẫn của ảnh), label (danh sách đường dẫn của ảnh phân đoạn), w và h (kích thước của ảnh).
    #Các tham số này được gán cho các thuộc tính self.data, self.label, self.w, và self.h tương ứng.
    def __init__(self, data, label, w, h):
        # the paths of images
        self.data = data
        # the paths of segmentation images
        self.label = label
        self.w = w
        self.h = h
    #Phương thức __len__ trả về kích thước của tập dữ liệu bằng cách lấy độ dài của danh sách self.data.
    def __len__(self):
        return len(self.data)

    def __getitem__(self, i): #Phương thức __getitem__ nhận một tham số i và trả về một mẫu dữ liệu tại vị trí index i
        # read data
        #đọc ảnh từ đường dẫn image_path + '/' + str(self.data[i][0]) với kích thước mục tiêu (IMAGE_W, IMAGE_H, 3) bằng cách sử dụng image.load_img
        img = image.load_img(image_path + '/' + str(self.data[i][0]),target_size=(IMAGE_W,IMAGE_H,3))
        #chuyển đổi ảnh sang mảng numpy bằng image.img_to_array và tiến hành chuẩn hóa giá trị pixel trong khoảng [0, 1] bằng cách chia cho 255
        img = image.img_to_array(img)
        img = img /255
        #lấy nhãn phân đoạn tương ứng từ danh sách self.label và trả về cả ảnh và nhãn dưới dạng tuple.
        label = self.label[i]
        return img, label


In [25]:
y = np.array(train.drop(columns=["Filenames"])) #tạo ra một DataFrame mới từ train bằng cách loại bỏ cột có tên "Filenames".
X = np.array(train) #chuyển đổi DataFrame đó thành một mảng NumPy. Kết quả được gán cho biến y

In [26]:
# và y là các mảng NumPy chứa dữ liệu đầu vào và nhãn tương ứng
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1) #gọi hàm train_test_split
#X và y là dữ liệu đầu vào và nhãn cần chia.
#random_state=42 đặt một giá trị seed để đảm bảo tính lặp lại trong quá trình chia ngẫu nhiên.
#test_size=0.1 nghĩa là 10% dữ liệu sẽ được sử dụng làm tập kiểm tra, còn lại 90% là tập huấn luyện.
#Hàm train_test_split trả về bốn mảng NumPy:
#X_train: Dữ liệu đầu vào cho tập huấn luyện.
#X_test: Dữ liệu đầu vào cho tập kiểm tra.
#y_train: Nhãn cho tập huấn luyện.
#y_test: Nhãn cho tập kiểm tra.

In [27]:
# Xay dung dataset va Dataloader
# Build dataaset
train_dataset = Dataset(X_train, y_train, IMAGE_W, IMAGE_H) #Tạo một đối tượng Dataset mới với tên train_dataset cho tập dữ liệu huấn luyện.
test_dataset = Dataset(X_test, y_test, IMAGE_W, IMAGE_H) #tạo một đối tượng Dataset mới với tên test_dataset cho tập dữ liệu kiểm tra.

# Loader
#Tạo một đối tượng Dataloader mới với tên train_loader cho tập dữ liệu huấn luyện.
train_loader = Dataloader(train_dataset, 64, len(train_dataset)) #64 là kích thước của batch, len(train_dataset) là kích thước của tập dữ liệu huấn luyện.
test_loader = Dataloader(test_dataset, 64, len(test_dataset)) #tạo một đối tượng Dataloader mới với tên test_loader cho tập dữ liệu kiểm tra.
#train_loader và test_loader sẽ được sử dụng trong quá trình huấn luyện và đánh giá mô hình. Cung cấp cách tải dữ liệu hiệu quả bằng cách chia thành các batch, giúp tiết kiệm bộ nhớ và tăng tốc độ xử lý.

In [ ]:
# Use fit method instead of fit_generator
model.fit(train_loader, epochs=20, validation_data=test_loader) #epochs=10 chỉ định số lần lặp lại quá trình huấn luyện trên toàn bộ dữ liệu huấn luyện.
model.save("/content/gdrive/MyDrive/Dataset/luanai_dataset/model.h5")
#validation_data=test_loader cung cấp dữ liệu kiểm tra cho mô hình để đánh giá sau mỗi epoch. test_loader là một đối tượng Dataloader chứa dữ liệu kiểm tra.

In [ ]:
tf.keras.models.load_model("/content/gdrive/MyDrive/Dataset/luanai_dataset/model.h5")

In [ ]:
img = image.load_img("/content/gdrive/MyDrive/Dataset/luanai_dataset/test/img-test01.jpg", target_size=(IMAGE_W,IMAGE_H,3)) #target_size=(IMAGE_W, IMAGE_H, 3) chỉ định kích thước mục tiêu của ảnh đầu vào.
img = image.img_to_array(img) #Chuyển đổi đối tượng ảnh img thành một mảng NumPy bằng cách sử dụng hàm image.img_to_array.
img = img/255 #Chuẩn hóa giá trị pixel của ảnh trong khoảng từ 0 đến 1 bằng cách chia cho 255 (giá trị pixel tối đa).
#Thêm một chiều mới vào đầu mảng img bằng cách sử dụng hàm np.expand_dims.
tensor = np.expand_dims(img, axis=0) #axis=0 chỉ định rằng chiều mới sẽ được thêm vào đầu
#Kết quả là một tensor 4 chiều với hình dạng (1, IMAGE_H, IMAGE_W, 3), trong đó 1 là kích thước batch.

In [ ]:
#tensor là tensor ảnh đầu vào đã được chuẩn bị sẵn sàng cho việc dự đoán.
y_pred = model.predict(tensor)#predict là một phương thức của mô hình, cho phép dự đoán nhãn hoặc giá trị đầu ra cho một tensor đầu vào.
y_pred #in ra giá trị của y_pred để kiểm tra kết quả dự đoán.

In [ ]:
classes = np.array(train.columns[1:])

top_3 = np.argsort(y_pred[0])[:-4:-1]
top_3

In [ ]:
for i in range(3):
    print("{}".format(classes[top_3[i]])+" ({:.3})".format(y_pred[0][top_3[i]] * 100))
plt.imshow(img)

In [ ]:
img = image.load_img("/content/gdrive/MyDrive/Dataset/luanai_dataset/test/img-test02.jpg", target_size=(IMAGE_W, IMAGE_H, 3))
img = image.img_to_array(img) / 255
tensor = np.expand_dims(img, axis=0)

y_pred = model.predict(tensor)
classes = np.array(train.columns[1:])
top_3 = np.argsort(y_pred[0])[:-4:-1]

for i in range(3):
    print("{}".format(classes[top_3[i]]) + " ({:.3})".format(y_pred[0][top_3[i]] * 100))
plt.imshow(img)

In [ ]:
img = image.load_img("/content/gdrive/MyDrive/Dataset/luanai_dataset/test/img-test03.jpg", target_size=(IMAGE_W, IMAGE_H, 3))
img = image.img_to_array(img) / 255
tensor = np.expand_dims(img, axis=0)

y_pred = model.predict(tensor)
classes = np.array(train.columns[1:])
top_3 = np.argsort(y_pred[0])[:-4:-1]

for i in range(3):
    print("{}".format(classes[top_3[i]]) + " ({:.3})".format(y_pred[0][top_3[i]] * 100))
plt.imshow(img)

In [ ]:
img = image.load_img("/content/gdrive/MyDrive/Dataset/luanai_dataset/test/img-test04.jpg", target_size=(IMAGE_W, IMAGE_H, 3))
img = image.img_to_array(img) / 255
tensor = np.expand_dims(img, axis=0)

y_pred = model.predict(tensor)
classes = np.array(train.columns[1:])
top_3 = np.argsort(y_pred[0])[:-4:-1]

for i in range(3):
    print("{}".format(classes[top_3[i]]) + " ({:.3})".format(y_pred[0][top_3[i]] * 100))
plt.imshow(img)